# Simple analysis of GB1900

TODO:

- [ ] Histogram 2D using geopandas
- [ ] Legend beautification
- [ ] local_authority labels on the maps (when hover over them)
- [ ] change basemap to exclude Northern Ireland
- [ ] query maps based on lat/lon of pins, possibly add a new column with link to sheets
- [ ] abbreviation disambiguation ---> remove dots (in search/string matching step)

In [ ]:
# %matplotlib notebook 

## Read the Gazetteer (gb1900_gazetteer_complete_july_2018.csv)

In [ ]:
import itertools
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd

In [ ]:
with open("./gb1900_gazetteer_complete_july_2018.csv", encoding='UTF-16') as f:
    df = pd.read_csv(f)

In [ ]:
df.head()

## Find and group unique text strings

In [ ]:
min_counts = 5000
xy2plot = df.groupby("final_text").count().sort_values("pin_id", ascending=False)["pin_id"]
xy2plot_filtered = xy2plot[xy2plot > min_counts]
print("Total number of unique text strings: {}".format(len(xy2plot)))
print("Number of unique text strings (after filtering): {}".format(len(xy2plot_filtered)))

In [ ]:
ax = plt.figure(figsize=(15, 7)).add_subplot(111)
xy2plot_filtered.plot(ax=ax, kind='bar')
ax.set_xlabel("Place", size=24)
ax.set_ylabel("Count", size=24)
ax.tick_params(labelsize=14)
ax.grid()
plt.show()

In [ ]:
# list text/number of xy2plot_filtered
first_index = 0
last_index = 50
xy2plot_filtered[first_index:last_index]

In [ ]:
# one instance with longitude < -10
df[df["longitude"] < -10]

## Select texts/lables for further analysis/plotting

In [ ]:
# read OS_Abbreviations.xlsx file which contains mapping between label and text
os_abbreviation = pd.read_excel("./OS_Abbreviations.xlsx")

search_string = "engine"
list_of_selected_text = os_abbreviation[os_abbreviation["text"].str.contains(search_string, 
                                                                             case=False, 
                                                                             na=False)].label.to_list()
list_of_selected_text.extend([search_string])
print("Here is the list of selected texts that contain {}: {}".format(search_string, list_of_selected_text))

for itext in list_of_selected_text:
    df_filtered = df[df["final_text"].str.contains(itext, case=False, na=False)]
    print(10*"=" + " {}. Total: {}".format(itext,df_filtered.shape[0]))
    print(df_filtered.groupby("final_text").count().sort_values("pin_id", ascending=False)["pin_id"][0:60])

In [ ]:
# Complex search: 2 strings (AND and NOT)

search_string_1 = "mill"
search_string_2 = "disused"
list_of_selected_text_1 = os_abbreviation[os_abbreviation["text"].str.contains(search_string_1, case=False, na=False)].label.to_list()
list_of_selected_text_2 = os_abbreviation[os_abbreviation["text"].str.contains(search_string_2, case=False, na=False)].label.to_list()    
list_of_selected_text_1.extend([search_string_1])
list_of_selected_text_2.extend([search_string_2])

print("Here is the list of selected texts that contain {}: {}".format(search_string_1, list_of_selected_text_1))
print("Here is the list of selected texts that contain {}: {}".format(search_string_2, list_of_selected_text_2))

# contains both search_string_1 and search_string_2 eg. 'mill & disused'
for itext_1 in list_of_selected_text_1:
    df_filtered = df[(df["final_text"].str.contains(itext_1, case=False, na=False))]
    for itext_2 in list_of_selected_text_2:
        print()
        df_filtered = df_filtered[(df_filtered["final_text"].str.contains(itext_2, case=False, na=False))]
        print(10*"=" + " {} and {}. Total: {}".format(itext_1, itext_2, df_filtered.shape[0]))
        print(df_filtered.groupby("final_text").count().sort_values("pin_id", ascending=False)["pin_id"][0:60])
    
# contains search_string_1 and NOT search_string_2 eg. 'mine' but not 'mineral'
# for itext_1 in list_of_selected_text_1:
#     df_filtered = df[(df["final_text"].str.contains(itext_1, case=False, na=False))]
#     for itext_2 in list_of_selected_text_2:
#         df_filtered = df_filtered[(~df_filtered["final_text"].str.contains(itext_2, case=False, na=False))]
#         print()
#         print(10*"=" + " {} not {}. Total: {}".format(itext_1, itext_2, df_filtered.shape[0]))
#         print(df_filtered.groupby("final_text").count().sort_values("pin_id", ascending=False)["pin_id"][0:60])

In [ ]:
#1: each text/label is treated as an individual label
#2: combine list_of_selected_text
text_method = 2

# enter selected texts here
list_of_selected_text = ["School", "Sch."]

lats2plot = []
lons2plot = []
for i in list_of_selected_text:
    df_filtered = df[df["final_text"].isin([i])]
    print("'{}' has {} rows.".format(i, len(df_filtered)))
    lats2plot.append(df_filtered["latitude"].tolist())
    lons2plot.append(df_filtered["longitude"].tolist())

if text_method == 2:
    lats2plot = [list(itertools.chain.from_iterable(lats2plot))]
    lons2plot = [list(itertools.chain.from_iterable(lons2plot))]
    list_of_selected_text = [", ".join(list_of_selected_text)]    

## GeoPandas for plotting

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Download shapefiles from earthworks.stanford.edu
world = gpd.read_file("./data_EPSG_4326/GBR_adm0.shp")

### Plot labels separately (text_method = 1)

In [ ]:
list_colors = ["r", "b", "g", "c", "m"]

for igrp in range(len(lats2plot)):
    geometry = [Point(xy) for xy in zip(lons2plot[igrp], lats2plot[igrp])]
    gdf = gpd.GeoDataFrame(geometry=geometry)
    gdf.plot(ax=world.plot(figsize=(20, 20), edgecolor='k', color='none'), 
             marker='o', 
             color=list_colors[igrp], 
             markersize=2,
             alpha=0.2
            )
    plt.title(list_of_selected_text[igrp], size=24, weight='bold')
    plt.xlim(xmin=-10)
    plt.legend(prop={'size': 32})
    plt.grid()
plt.show()

### Plot labels together on one map (text_method = 1 or 2)

In [ ]:
list_colors = ["r", "b", "g", "c", "m"]

fig, ax = plt.subplots(1, 1, figsize=(20, 20))

world.plot(ax=ax, edgecolor='k', color='none')

for igrp in range(len(lats2plot)):
    geometry = [Point(xy) for xy in zip(lons2plot[igrp], lats2plot[igrp])]
    gdf = gpd.GeoDataFrame(geometry=geometry)
    gdf.plot(ax=ax, 
             marker='o', 
             color=list_colors[igrp], 
             markersize=2,
             label=list_of_selected_text[igrp],
             alpha=0.2
            )
plt.xlim(xmin=-10)
plt.grid()
plt.legend(prop={'size': 32})
plt.show()

### Simple plot (no projection)

In [ ]:
list_colors = ["r", "b", "g", "c", "m"]
fig, ax = plt.subplots(1, 1, figsize=(20, 20))

for igrp in range(len(lats2plot)):
    plt.scatter(np.array(lons2plot[igrp]), 
            np.array(lats2plot[igrp]), 
            c=list_colors[igrp],
            s=3,
            edgecolors='none',
            alpha=0.4,
            label=list_of_selected_text[igrp])
    
plt.xlim(xmin=-10)
plt.grid()
plt.legend(prop={'size': 32})
plt.show()

## 2D Histograms

In [ ]:
list_colors = ["r", "b", "g", "c", "m"]
plt.figure(figsize=(20, 20))

igrp = 0
plt.hist2d(np.array(lons2plot[igrp]), 
           np.array(lats2plot[igrp]),
           bins=(100, 100),
           range=[[-8, 2], [50, 62]],
           cmap="Greys",
           label=list_of_selected_text[igrp],
           #cmin=1,
           #cmax=100,
           norm=LogNorm()
          )

plt.title(list_of_selected_text[igrp], size=24, weight='bold')
plt.xlim(xmin=-10)
plt.legend(prop={'size': 32})
plt.colorbar()
plt.grid()
plt.show()

## Simple statistical analysis

In [ ]:
x_bin = 100
y_bin = 100

igrp = 0
hist_map = np.histogram2d(np.array(lons2plot[igrp]), 
                          np.array(lats2plot[igrp]), 
                          bins=(x_bin, y_bin))

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(hist_map[1][:-1], hist_map[0].sum(axis=1), lw=2, c='k')
plt.ylabel("Frequency", size=20)
plt.xlabel("Longitude", size=20)
plt.title(list_of_selected_text[igrp], size=24, weight='bold')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(hist_map[2][:-1], hist_map[0].sum(axis=0), lw=2, c='k')
plt.ylabel("Frequency", size=20)
plt.xlabel("Latitude", size=20)
plt.grid()

## Subtract two histograms

In [ ]:
positive_list_selected_text = ["School", "Sch."]
negative_list_selected_text = ["Chap.", "Chapel"]
x_bin = 100
y_bin = 100

# ============ positive
lats2plot = []
lons2plot = []
for i in positive_list_selected_text:
    df_filtered = df[df["final_text"].isin([i])]
    print("'{}' has {} rows.".format(i, len(df_filtered)))
    lats2plot.extend(df_filtered["latitude"].tolist())
    lons2plot.extend(df_filtered["longitude"].tolist())
lats2plot = [lats2plot]
lons2plot = [lons2plot]

hist_map_positive = np.histogram2d(np.array(lons2plot[0]), np.array(lats2plot[0]), bins=(x_bin, y_bin))

# ============ negative
lats2plot = []
lons2plot = []
for i in negative_list_selected_text:
    df_filtered = df[df["final_text"].isin([i])]
    print("'{}' has {} rows.".format(i, len(df_filtered)))
    lats2plot.extend(df_filtered["latitude"].tolist())
    lons2plot.extend(df_filtered["longitude"].tolist())
lats2plot = [lats2plot]
lons2plot = [lons2plot]

hist_map_negative = np.histogram2d(np.array(lons2plot[0]), np.array(lats2plot[0]), bins=(x_bin, y_bin))

In [ ]:
list_colors = ["r", "b", "g", "c", "m"]
plt.figure(figsize=(20, 20))

plt.pcolor(hist_map_positive[1], 
           hist_map_positive[2], 
           (hist_map_positive[0] - hist_map_negative[0]).T, 
           cmap="seismic",
           vmin=-50,
           vmax=50)
plt.xlim(xmin=-10)
plt.legend(prop={'size': 32})
plt.colorbar()
plt.grid()
plt.show()